In [24]:
import pandas as pd
import matplotlib 
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

In [2]:
df = pd.read_csv(r"./dataset.csv")

In [3]:
df['DATE OCC'] = pd.to_datetime(df['DATE OCC'], format='mixed')
df['Year OCC'] = df['DATE OCC'].dt.year
df['Month OCC'] = df['DATE OCC'].dt.month
df['DAY OCC'] = df['DATE OCC'].dt.day
df['DAY WEEK OCC'] = df['DATE OCC'].dt.day_name()

df.drop(columns=['DATE OCC'], inplace=True)
df.head()

,DR_NO,Date Rptd,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,Mocodes,...,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON,Year OCC,Month OCC,DAY OCC,DAY WEEK OCC
0,190326475,03/01/2020 0:00,2130,7,Wilshire,784,1,510,VEHICLE - STOLEN,NaN,...,NaN,NaN,1900 S LONGWOOD AV,NaN,34.0375,-118.3506,2020,3,1,Sunday
1,200106753,02/09/2020 0:00,1800,1,Central,182,1,330,BURGLARY FROM VEHICLE,1822 1402 0344,...,NaN,NaN,1000 S FLOWER ST,NaN,34.0444,-118.2628,2020,2,8,Saturday
2,200320258,11/11/2020 0:00,1700,3,Southwest,356,1,480,BIKE - STOLEN,0344 1251,...,NaN,NaN,1400 W 37TH ST,NaN,34.0210,-118.3002,2020,11,4,Wednesday
3,200907217,05/10/2023 0:00,2037,9,Van Nuys,964,1,343,SHOPLIFTING-GRAND THEFT ($950.01 & OVER),0325 1501,...,NaN,NaN,14000 RIVERSIDE DR,NaN,34.1576,-118.4387,2020,3,10,Tuesday
4,220614831,08/18/2022 12:00:00 AM,1200,6,Hollywood,666,2,354,THEFT OF IDENTITY,1822 1501 0930 2004,...,NaN,NaN,1900 TRANSIENT,NaN,34.0944,-118.3277,2020,8,17,Monday


In [4]:
df.drop(columns=['Date Rptd'], inplace=True)
df.head()

,DR_NO,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,Mocodes,Vict Age,...,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON,Year OCC,Month OCC,DAY OCC,DAY WEEK OCC
0,190326475,2130,7,Wilshire,784,1,510,VEHICLE - STOLEN,NaN,0,...,NaN,NaN,1900 S LONGWOOD AV,NaN,34.0375,-118.3506,2020,3,1,Sunday
1,200106753,1800,1,Central,182,1,330,BURGLARY FROM VEHICLE,1822 1402 0344,47,...,NaN,NaN,1000 S FLOWER ST,NaN,34.0444,-118.2628,2020,2,8,Saturday
2,200320258,1700,3,Southwest,356,1,480,BIKE - STOLEN,0344 1251,19,...,NaN,NaN,1400 W 37TH ST,NaN,34.0210,-118.3002,2020,11,4,Wednesday
3,200907217,2037,9,Van Nuys,964,1,343,SHOPLIFTING-GRAND THEFT ($950.01 & OVER),0325 1501,19,...,NaN,NaN,14000 RIVERSIDE DR,NaN,34.1576,-118.4387,2020,3,10,Tuesday
4,220614831,1200,6,Hollywood,666,2,354,THEFT OF IDENTITY,1822 1501 0930 2004,28,...,NaN,NaN,1900 TRANSIENT,NaN,34.0944,-118.3277,2020,8,17,Monday


In [5]:
print(df.dtypes)

DR_NO               int64
TIME OCC            int64
AREA                int64
AREA NAME          object
Rpt Dist No         int64
Part 1-2            int64
Crm Cd              int64
Crm Cd Desc        object
Mocodes            object
Vict Age            int64
Vict Sex           object
Vict Descent       object
Premis Cd         float64
Premis Desc        object
Weapon Used Cd    float64
Weapon Desc        object
Status             object
Status Desc        object
Crm Cd 1          float64
Crm Cd 2          float64
Crm Cd 3          float64
Crm Cd 4          float64
LOCATION           object
Cross Street       object
LAT               float64
LON               float64
Year OCC            int32
Month OCC           int32
DAY OCC             int32
DAY WEEK OCC       object
dtype: object


In [6]:
#removes duplicate rows according to "event_unique_id"
print('Original Data Size after dropping Duplicates')
df = df.drop_duplicates(subset='DR_NO',keep='first')
df.shape

Original Data Size after dropping Duplicates


(932140, 30)

In [7]:
# Group all the assualts
df.loc[df['Crm Cd Desc'].str.contains('ASSAULT'), 'Crm Cd Desc'] = 'ASSAULT'

In [8]:
drop_colmns = ['DR_NO', 'AREA', 'Rpt Dist No','Part 1-2','Mocodes','Weapon Used Cd','Premis Cd','Crm Cd 1','Crm Cd 2', 'Crm Cd 3', 'Crm Cd 4', 'DAY OCC','Status Desc', 'Cross Street']
df_dropped = df.drop(columns=drop_colmns)
df_dropped.head()

,TIME OCC,AREA NAME,Crm Cd,Crm Cd Desc,Vict Age,Vict Sex,Vict Descent,Premis Desc,Weapon Desc,Status,LOCATION,LAT,LON,Year OCC,Month OCC,DAY WEEK OCC
0,2130,Wilshire,510,VEHICLE - STOLEN,0,M,O,STREET,NaN,AA,1900 S LONGWOOD AV,34.0375,-118.3506,2020,3,Sunday
1,1800,Central,330,BURGLARY FROM VEHICLE,47,M,O,BUS STOP/LAYOVER (ALSO QUERY 124),NaN,IC,1000 S FLOWER ST,34.0444,-118.2628,2020,2,Saturday
2,1700,Southwest,480,BIKE - STOLEN,19,X,X,"MULTI-UNIT DWELLING (APARTMENT, DUPLEX, ETC)",NaN,IC,1400 W 37TH ST,34.0210,-118.3002,2020,11,Wednesday
3,2037,Van Nuys,343,SHOPLIFTING-GRAND THEFT ($950.01 & OVER),19,M,O,CLOTHING STORE,NaN,IC,14000 RIVERSIDE DR,34.1576,-118.4387,2020,3,Tuesday
4,1200,Hollywood,354,THEFT OF IDENTITY,28,M,H,SIDEWALK,NaN,IC,1900 TRANSIENT,34.0944,-118.3277,2020,8,Monday


In [9]:
df['Crm Cd Desc'].value_counts()

Crm Cd Desc
ASSAULT                                               194503
VEHICLE - STOLEN                                      100225
THEFT OF IDENTITY                                      57539
BURGLARY FROM VEHICLE                                  57538
BURGLARY                                               57015
                                                       ...  
BLOCKING DOOR INDUCTION CENTER                             6
FIREARMS EMERGENCY PROTECTIVE ORDER (FIREARMS EPO)         5
DISHONEST EMPLOYEE ATTEMPTED THEFT                         4
FIREARMS RESTRAINING ORDER (FIREARMS RO)                   4
TRAIN WRECKING                                             1
Name: count, Length: 132, dtype: int64

In [10]:
df_grouped = df_dropped.groupby(df_dropped['Year OCC'])

In [11]:
#Analysis by year
df_2020 = df_grouped.get_group(2020)
df_2021 = df_grouped.get_group(2021)
df_2022 = df_grouped.get_group(2022)
df_2023 = df_grouped.get_group(2023)
df_2024 = df_grouped.get_group(2024)

In [12]:
df_2020_grouped = df_2020.groupby(df_2020['Crm Cd Desc']).count()
# df_2021_grouped = df_2021.groupby(df_2021['Crm Cd Desc']).count()
# df_2022_grouped = df_2022.groupby(df_2022['Year OCC']).count()
# df_2023_grouped = df_2023.groupby(df_2023['Year OCC']).count()
# df_2024_grouped = df_2024.groupby(df_2024['Year OCC']).count()
# df_2020_grouped

In [13]:
plot = df_2020_grouped.iloc[:,0]
plot = pd.DataFrame(plot)
plot.columns = ['Number of Cases']
ax = plot.plot(kind='barh',figsize=(100,100),title='Number of Crimes in LA 2015')

In [14]:
df.head()

,DR_NO,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,Mocodes,Vict Age,...,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON,Year OCC,Month OCC,DAY OCC,DAY WEEK OCC
0,190326475,2130,7,Wilshire,784,1,510,VEHICLE - STOLEN,NaN,0,...,NaN,NaN,1900 S LONGWOOD AV,NaN,34.0375,-118.3506,2020,3,1,Sunday
1,200106753,1800,1,Central,182,1,330,BURGLARY FROM VEHICLE,1822 1402 0344,47,...,NaN,NaN,1000 S FLOWER ST,NaN,34.0444,-118.2628,2020,2,8,Saturday
2,200320258,1700,3,Southwest,356,1,480,BIKE - STOLEN,0344 1251,19,...,NaN,NaN,1400 W 37TH ST,NaN,34.0210,-118.3002,2020,11,4,Wednesday
3,200907217,2037,9,Van Nuys,964,1,343,SHOPLIFTING-GRAND THEFT ($950.01 & OVER),0325 1501,19,...,NaN,NaN,14000 RIVERSIDE DR,NaN,34.1576,-118.4387,2020,3,10,Tuesday
4,220614831,1200,6,Hollywood,666,2,354,THEFT OF IDENTITY,1822 1501 0930 2004,28,...,NaN,NaN,1900 TRANSIENT,NaN,34.0944,-118.3277,2020,8,17,Monday


In [15]:

col_list = ['Year OCC', 'Month OCC', 'DAY OCC', 'DAY WEEK OCC', 'TIME OCC', 'Crm Cd Desc', 'AREA NAME', 'LOCATION', 'Premis Desc']


df2 = df[col_list]
df2 = df2[df2['Year OCC'] > 2019]

#Factorize dependent variable column:
crime_var = pd.factorize(df2['Crm Cd Desc'])
df2['Crm Cd Desc'] = crime_var[0]
definition_list_MCI = crime_var[1]

#factorize independent variables:
premise_var = pd.factorize(df2['Premis Desc'])
df2['Premis Desc'] = premise_var[0]
definition_list_premise = premise_var[1] 

#factorize occurenceyear:
year_var = pd.factorize(df2['Year OCC'])
df2['Year OCC'] = year_var[0]
definition_list_year = year_var[1] 

#factorize occurencemonth:
month_var = pd.factorize(df2['Month OCC'])
df2['Month OCC'] = month_var[0]
definition_list_month = month_var[1] 

#factorize occurenceday:
day_var = pd.factorize(df2['DAY OCC'])
df2['DAY OCC'] = day_var[0]
definition_list_day = day_var[1] 

#factorize occurencedayofweek:
dayweek_var = pd.factorize(df2['DAY WEEK OCC'])
df2['DAY WEEK OCC'] = dayweek_var[0]
definition_list_day = dayweek_var[1] 

#factorize division:
division_var = pd.factorize(df2['AREA NAME'])
df2['AREA NAME'] = division_var[0]
definition_list_division = division_var[1] 

#factorize HOOD_ID:
hood_var = pd.factorize(df2['LOCATION'])
df2['LOCATION'] = hood_var[0]
definition_list_hood = hood_var[1] 

#factorize occurencehour:
hour_var = pd.factorize(df2['TIME OCC'])
df2['TIME OCC'] = hour_var[0]
definition_list_hour = hour_var[1] 


In [16]:
df2.head()

,Year OCC,Month OCC,DAY OCC,DAY WEEK OCC,TIME OCC,Crm Cd Desc,AREA NAME,LOCATION,Premis Desc
0,0,0,0,0,0,0,0,0,0
1,0,1,1,1,1,1,1,1,1
2,0,2,2,2,2,2,2,2,2
3,0,0,3,3,3,3,3,3,3
4,0,3,4,4,4,4,4,4,4


In [20]:
x = df2.drop(['Crm Cd Desc'],axis=1).values
y = df2['Crm Cd Desc'].values
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, random_state = 21)
binary_encoder = OneHotEncoder(categories='auto')
encoded_X = binary_encoder.fit_transform(x)
X_train_OH, X_test_OH, y_train_OH, y_test_OH = train_test_split(encoded_X, y, test_size = 0.25, random_state = 21)
classifier = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 42)

classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
unique_classes = np.unique(np.concatenate((y_test, y_pred)))
target_names = [definition_list_MCI[i] for i in unique_classes]

print("Accuracy of Random Forest : ",accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred)) 
# print(classification_report(y_test,y_pred, target_names=definition_list_MCI))
print(classification_report(y_test, y_pred, target_names=target_names))

In [27]:
classifier = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 42)
classifier.fit(X_train_OH, y_train_OH)
y_pred_OH = classifier.predict(X_test_OH)

print("Accuracy of Random Forest with OneHotEncoder : ",accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test_OH, y_pred_OH)) 
print(classification_report(y_test_OH,y_pred_OH, target_names=definition_list_MCI))